## Objectives

After completing this lab you will be able to:

*   Read CSV and JSON file types.
*   Extract data from the above file types.
*   Transform data.
*   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


In [2]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2023-04-01 09:39:18--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip’

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2023-04-01 09:39:18 (267 MB/s) - ‘source.zip’ saved [2707/2707]



mkdir: missing operand
Try 'mkdir --help' for more information.


In [5]:
!unzip source.zip

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


In [6]:

a = pd.read_csv("/content/source1.csv")


In [7]:
a.head()

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


In [9]:
a.shape

(5, 3)

In [10]:
b = pd.read_json("/content/source1.json", lines = True)

In [11]:
b.head()

,name,height,weight
0,jack,68.70,123.30
1,tom,69.80,141.49
2,tracy,70.01,136.46
3,john,67.90,112.37


In [12]:
tree = ET.parse("/content/source1.xml")

In [14]:
c = tree.getroot()

In [15]:
c

<Element 'data' at 0x7f83e73432c0>

In [16]:
dataframe = pd.DataFrame(columns = ["name", "height","weight"])
for person in c:
    name = person.find("name").text
    height = float(person.find("height").text)
    weight = float(person.find("weight").text)
    dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index = True)


<ipython-input-16-fa5559c60bd4>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index = True)
<ipython-input-16-fa5559c60bd4>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index = True)
<ipython-input-16-fa5559c60bd4>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index = True)
<ipython-input-16-fa5559c60bd4>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.a

In [17]:
dataframe

,name,height,weight
0,simon,67.90,112.37
1,jacob,66.78,120.67
2,cindy,66.49,127.45
3,ivan,67.62,114.14


SET PATHS


In [18]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

EXTRACT

CSV EXTRACT FUNCTION

In [19]:
def extract_from_csv(file_to_process):
  dataframe = pd.read_csv(file_to_process)
  return dataframe

JSON EXTRACT FUNCTION

In [20]:
def extract_from_json(file_to_process):
  dataframe = pd.read_json(file_to_process, lines = True)
  return dataframe
  

XML Extract Function

XML is a software- and hardware-independent tool for storing and transporting data.

XML stands for eXtensible Markup Language
XML is a markup language much like HTML
XML was designed to store and transport data
XML was designed to be self-descriptive
XML is a W3C Recommendation

In [21]:
def extract_from_xml(file_to_process):
  dataframe = pd.DataFrame(columns = ["name", "height","weight"])
  tree = ET.parse(file_to_process)
  root = tree.getroot()
  for person in root:
    name = person.find("name").text
    height = float(person.find("height").text)
    weight = float(person.find("weight").text)
    dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index = True)
  return dataframe  


### Extract Function

In [22]:
def extract():
  extracted_data = pd.DataFrame(columns = ['name', 'height', 'weight']) # create an empty data frame to hold extracted data 
  #process all csv files
  for csvfile in glob.glob("*.csv"):
    extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index = True)
  #process all json files
  for jsonfile in glob.glob("*.json"):
    extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index = True)

  #process all xml files
  for xmlfile in glob.glob("*.xml"):
    extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index = True)

  
  return extracted_data

  
  





***TRANSFORM***

The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms

In [24]:
def transform(data):
  #convert height which is in inches to millimeter
  #convert the data type of the column into float
  #data.height = data.height.astype(float)
  #convert inches to meters and round off to two decimal places (one inch is 0.0254 meters)
  data['height'] = round(data.height *0.0254, 2)

  #Convert weight which is in pounds to kilograms
  #Convert the datatype of the column into float
  #data.weight = data.weight.astype(float)
  #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)

  data['weight'] = round(data.weight * 0.45359237,2)
  return data
        


**LOADING**

In [25]:
def load(targetfile, data_to_load):
  data_to_load.to_csv(targetfile)

**LOGGING**

In [26]:
def log(message):
  timestamp_format = '%Y-%h-%d-%H:%M:%S' #Year-Monthname-Day-Hour-Minute-Second
  now=datetime.now() #get current timestamp
  timestamp = now.strftime(timestamp_format)
  with open("logfile.txt","a") as f:
    f.write(timestamp +','+ message+'\n')
    


**RUNNING ETL PROCESS**

In [27]:
log("ETL JOB STARTED")

In [28]:
f = open("logfile.txt", "r")
print(f.read())

2023-Apr-01-10:11:00,ETL JOB STARTED



In [29]:
log("Extracted Phase Started")
extracted_data = extract()
log("Extract Phase Ended")

<ipython-input-22-248bc65cae0b>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index = True)
<ipython-input-22-248bc65cae0b>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index = True)
<ipython-input-22-248bc65cae0b>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index = True)
<ipython-input-22-248bc65cae0b>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(js

In [30]:
f = open("logfile.txt", "r")
print(f.read())

2023-Apr-01-10:11:00,ETL JOB STARTED
2023-Apr-01-10:16:17,Extracted Phase Started
2023-Apr-01-10:16:17,Extract Phase Ended



In [31]:
extracted_data

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,alex,65.78,112.99
6,ajay,71.52,136.49
7,alice,69.40,153.03
8,ravi,68.22,142.34
9,joe,67.79,144.30


In [33]:
log("Transform Phase Started")
transformed_data = transform(extracted_data)
log("Transform Phase Ended")

In [34]:
f = open("logfile.txt", "r")
print(f.read())

2023-Apr-01-10:11:00,ETL JOB STARTED
2023-Apr-01-10:16:17,Extracted Phase Started
2023-Apr-01-10:16:17,Extract Phase Ended
2023-Apr-01-10:19:02,Load Phase Started
2023-Apr-01-10:20:59,Transform Phase Started
2023-Apr-01-10:20:59,Transform Phase Ended



In [35]:
transformed_data

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,alex,1.67,51.25
6,ajay,1.82,61.91
7,alice,1.76,69.41
8,ravi,1.73,64.56
9,joe,1.72,65.45


In [36]:
log("Load Phase Started")
load(targetfile, transformed_data)
log("Load Phase Ended")


In [37]:
log("ETL Job Ended")